# Setup

In [15]:
# Import needed libraries
import pandas as pd
import numpy as np
import nflreadpy as nfl

In [16]:
# Set display options
pd.set_option('display.max_rows', 1000)

# Initial data loading

In [17]:
team_stats = nfl.load_team_stats(True)
team_stats = team_stats.to_pandas()
team_stats

,season,week,team,season_type,opponent_team,completions,attempts,passing_yards,passing_tds,passing_interceptions,...,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance
0,1999,1,ARI,REG,PHI,25,48,274,1,3,...,1,1,0,0,1.0,1,1,0,0,31
1,1999,1,ATL,REG,MIN,18,31,290,1,0,...,2,2,0,0,1.0,0,0,0,0,0
2,1999,1,BUF,REG,IND,22,42,300,1,2,...,0,0,0,0,NaN,0,0,0,0,0
3,1999,1,CAR,REG,NO,16,32,207,1,1,...,1,1,0,0,1.0,0,0,0,0,0
4,1999,1,CHI,REG,KC,31,47,322,2,0,...,2,2,0,0,1.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14344,2025,13,SEA,REG,MIN,14,26,128,0,0,...,2,2,0,0,1.0,0,0,0,0,0
14345,2025,13,SF,REG,CLE,16,30,168,1,0,...,2,2,0,0,1.0,0,0,0,0,0
14346,2025,13,TB,REG,ARI,18,28,194,1,0,...,2,2,0,0,1.0,0,0,0,0,0
14347,2025,13,TEN,REG,JAX,24,38,141,0,0,...,0,0,0,0,NaN,0,0,0,0,0


# Preprocessing

In [18]:
def preprocess_team_data(df):
    '''
    -Preprocesses/cleans up the raw df from nfl.load_team_stats
    -Adds features for previous game, current season, and previous 
        season stats for both team and opponent

    Parameters:
    df - original df returned from nfl.load_team_stats

    Returns:
    df - data in clean form
    '''

    # # # # # # # # #
    # Preprocessing #
    # # # # # # # # #

    df = df.query('season >= 2007').copy()

    # Duplicate columns
    df = df.drop(['receiving_tds', 
                  'receiving_first_downs', 
                  'receiving_2pt_conversions', 
                  'receiving_air_yards', 
                  'receptions',
                  'receiving_yards'], axis=1)
    # Irrelevant columns
    df = df.drop(['fg_made_list',
                  'fg_missed_list',
                  'fg_blocked_list'], axis=1)
    
    # Define the points_scored column based on TDs, PATs, FGs, and 2pt conversions
    df['points_scored'] = (6 * (df['passing_tds'] + 
                                df['rushing_tds'] +
                                df['special_teams_tds'] +
                                df['def_tds']) + 
                           3 * df['fg_made'] +
                           2 * (df['passing_2pt_conversions'] +
                                df['rushing_2pt_conversions']) +
                           1 * df['pat_made'])
    
    # Add opponent_points_scored
    opp_points_scored_df = df[['season', 'week', 'team', 'points_scored']]
    opp_points_scored_df = opp_points_scored_df.rename(columns={'team': 'opponent_team', 
                                                                'points_scored': 'opponent_points_scored'})
    df = df.merge(opp_points_scored_df, on=['season', 'week', 'opponent_team'], how='inner')

    # target variable: 1 if win, 0.5 if draw, 0 is loss
    df['outcome'] = (np.sign(df['points_scored'] - df['opponent_points_scored']) / 2) + 0.5 
    
    # Make sure there's no duplicate (season, week, team) combinations
    if df.duplicated(subset=["season", "week", "team"]).any():
        raise ValueError('Error: duplicate games')

    # # # # # # # # #
    # Add Features  #
    # # # # # # # # #
    
    # Add previous game features
    cols = df.columns
    keep_cols = ['season', 'week', 'team', 'season_type', 'opponent_team', 'points_scored']
    prev_cols = [col for col in cols if col not in keep_cols] + ['points_scored']

    # Shift all the stats so that we only know last game's stats
    df = df.sort_values(by=['team', 'season', 'week']).reset_index(drop=True)
    prev_game_df = df.groupby('team')[prev_cols + ['week']].shift(1).add_prefix('previous_game_')
    df = pd.concat([df, prev_game_df], axis=1)

    # Add opponent prev_game info
    df = df.sort_values(by=['team', 'season', 'week']).reset_index(drop=True)
    opp_prev_game_df = df[['team', 'season', 'week'] + list(prev_game_df.columns)].copy()
    opp_prev_game_df = opp_prev_game_df.rename(columns={'team': 'opponent_team'})
    opp_prev_game_df = opp_prev_game_df.rename(columns=lambda x: x.replace('previous_game_', 'opponent_previous_game_') if x.startswith('previous_game_') else x)
    df = df.merge(opp_prev_game_df, on=['season', 'week', 'opponent_team'], how='inner')

    # Add current season averages for team and opponent
    # Team
    df = df.sort_values(by=['team', 'season', 'week']).reset_index(drop=True)
    df_copy = df.copy()
    team_season_avg_df = (df_copy
                          .groupby(['team', 'season'])[prev_cols]
                          .expanding()
                          .mean()
                          .shift(1)
                          .reset_index(drop=True)
                          .add_prefix('season_average_'))
    df.loc[df_copy.index, team_season_avg_df.columns] = team_season_avg_df

    # Opponent
    df = df.sort_values(by=['team', 'season', 'week']).reset_index(drop=True)
    df_copy = df.copy()
    df_copy['og_index'] = df_copy.index
    opp_season_avg_df = (df_copy
                        .groupby(['team', 'season'])[prev_cols]
                        .expanding()
                        .mean()
                        .shift(1)
                        .reset_index()).rename(columns={'team': 'opponent_team'})
    opp_season_avg_df = opp_season_avg_df.rename(columns=lambda x: ('opponent_season_average_'+x) if x in prev_cols else x)
    
    # remerge based on og_index to get week back
    opp_season_avg_df = opp_season_avg_df.merge(df_copy[['og_index', 'week']], 
                                                left_on=['level_2'],
                                                right_on=['og_index'],
                                                how='inner')
    opp_season_avg_df = opp_season_avg_df.drop(columns=['og_index', 'level_2'])
    df_copy = df_copy.drop(columns=['og_index'])

    df = df_copy.merge(opp_season_avg_df, 
                       on=['opponent_team', 'season', 'week'],
                       how='inner')

    # Add previous season averages for team and opponent
    # Team
    df = df.sort_values(by=['team', 'season', 'week']).reset_index(drop=True)
    team_prev_season_avg_df = (df
                               .groupby(['team', 'season'])[prev_cols]
                               .mean()
                               .groupby('team')
                               .shift(1)
                               .reset_index())
    team_prev_season_avg_df = team_prev_season_avg_df.rename(columns=lambda x: ('previous_season_average_'+x) if x in prev_cols else x)    
    df = df.merge(team_prev_season_avg_df, on=['team', 'season'], how='inner')
    
    # Opponent
    opp_prev_season_avg_df = (df_copy
                        .groupby(['team', 'season'])[prev_cols]
                        .mean()
                        .groupby('team')
                        .shift(1)
                        .reset_index()).rename(columns={'team': 'opponent_team'})
    opp_prev_season_avg_df = opp_prev_season_avg_df.rename(columns=lambda x: ('opponent_previous_season_average_'+x) if x in prev_cols else x)
    df = df.merge(opp_prev_season_avg_df, on=['opponent_team', 'season'], how='inner')

    # Add bye week info
    df['had_bye_week_last'] = np.where(df['week'] - df['previous_game_week'] == 1, 0, 1)
    df['opponent_had_bye_week_last'] = np.where(df['week'] - df['opponent_previous_game_week'] == 1, 0, 1)
    df = df.drop(columns=['previous_game_week', 'opponent_previous_game_week'])

    # # # # # # # # 
    # Handle NaNs #
    # # # # # # # # 
    df = df.query('season > 2007').copy()
    for c in ['fg_long', 'fg_pct', 'pat_pct']:
        # Fill in season_average vars
        last = f'season_average_{c}'
        new = f'previous_season_average_{c}'
        df[last] = df[last].fillna(df[new])

        # Fill in opponent_season_average vars
        last = f'opponent_season_average_{c}'
        new = f'opponent_previous_season_average_{c}'
        df[last] = df[last].fillna(df[new])

        # Fill in previous_game vars
        last = f'previous_game_{c}'
        new = f'season_average_{c}'
        df[last] = df[last].fillna(df[new])

        # Fill in opponent_previous_game vars
        last = f'opponent_previous_game_{c}'
        new = f'opponent_season_average_{c}'
        df[last] = df[last].fillna(df[new])


    # Drop same game stats - would make model omniscient
    df['y'] = df['outcome']
    drop_vars = prev_cols + ['opponent_points_scored']
    df = df.drop(columns=drop_vars)
    df = df.rename(columns={'y': 'outcome'})


    # # # # # # # # #
    # Check Output  #
    # # # # # # # # #
    if df.isna().sum().sum() != 0:
        raise ValueError("NaN Handling didn't work - still some NaNs left")
    
    df = df.sort_values(by=['season', 'week', 'team']).reset_index(drop=True)
    
    return df

In [19]:
df = preprocess_team_data(team_stats)

In [20]:
df

,season,week,team,season_type,opponent_team,previous_game_completions,previous_game_attempts,previous_game_passing_yards,previous_game_passing_tds,previous_game_passing_interceptions,...,opponent_previous_season_average_gwfg_att,opponent_previous_season_average_gwfg_missed,opponent_previous_season_average_gwfg_blocked,opponent_previous_season_average_gwfg_distance,opponent_previous_season_average_opponent_points_scored,opponent_previous_season_average_outcome,opponent_previous_season_average_points_scored,had_bye_week_last,opponent_had_bye_week_last,outcome
0,2008,1,ARI,REG,SF,23.0,39.0,300.0,3.0,2.0,...,0.062500,0.0625,0.000000,3.250000,22.375000,0.281250,13.187500,1,1,1.0
1,2008,1,ATL,REG,DET,17.0,27.0,251.0,4.0,0.0,...,0.062500,0.0000,0.000000,2.312500,27.250000,0.437500,21.125000,1,1,1.0
2,2008,1,BAL,REG,CIN,16.0,27.0,171.0,1.0,0.0,...,0.000000,0.0000,0.000000,0.000000,24.062500,0.437500,22.875000,1,1,1.0
3,2008,1,BUF,REG,SEA,16.0,30.0,133.0,0.0,0.0,...,0.000000,0.0000,0.000000,0.000000,18.722222,0.611111,24.444444,1,1,1.0
4,2008,1,CAR,REG,LAC,15.0,24.0,174.0,2.0,1.0,...,0.000000,0.0000,0.000000,0.000000,17.000000,0.684211,23.736842,1,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9607,2025,13,SEA,REG,MIN,16.0,26.0,244.0,2.0,0.0,...,0.055556,0.0000,0.000000,1.611111,19.166667,0.777778,24.166667,0,0,1.0
9608,2025,13,SF,REG,CLE,23.0,32.0,193.0,1.0,3.0,...,0.000000,0.0000,0.000000,0.000000,25.588235,0.176471,14.705882,0,0,1.0
9609,2025,13,TB,REG,ARI,17.0,34.0,103.0,1.0,2.0,...,0.117647,0.0000,0.000000,3.882353,22.294118,0.411765,22.588235,0,0,1.0
9610,2025,13,TEN,REG,JAX,28.0,42.0,256.0,1.0,0.0,...,0.058824,0.0000,0.000000,2.882353,25.470588,0.235294,18.470588,0,0,0.0


# Save data

In [21]:
df.to_csv('../data/clean_data.csv', index=False)